In [1]:
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns

TET2 EXPERIMENT

In [2]:
rsem_m1 = pd.read_csv('/work/iku/exp1019-cancer-epigenetics-and-ervs/01_BS_and_ERV/results/03_rsem/raw/TET2/1_RNA-Seq_SRR21643414.genes.results', sep = '\t')
rsem_m2 = pd.read_csv('/work/iku/exp1019-cancer-epigenetics-and-ervs/01_BS_and_ERV/results/03_rsem/raw/TET2/2_RNA-Seq_SRR21643410.genes.results', sep = '\t')
rsem_m3 = pd.read_csv('/work/iku/exp1019-cancer-epigenetics-and-ervs/01_BS_and_ERV/results/03_rsem/raw/TET2/3_RNA-Seq_SRR21643406.genes.results', sep = '\t')
rsem_m4 = pd.read_csv('/work/iku/exp1019-cancer-epigenetics-and-ervs/01_BS_and_ERV/results/03_rsem/raw/TET2/4_RNA-Seq_SRR21643402.genes.results', sep = '\t')
rsem_m5 = pd.read_csv('/work/iku/exp1019-cancer-epigenetics-and-ervs/01_BS_and_ERV/results/03_rsem/raw/TET2/5_RNA-Seq_SRR21643398.genes.results', sep = '\t')


In [3]:
columns = ['seqname', 'source', 'feature', 'start', 'end', 'score','strand','frame', 'attribute']

ensembl_gtf = pd.read_csv('/work/iku/exp1019-cancer-epigenetics-and-ervs/reference/GRCm38-102/ensembl+geve/GRCm38-102.with_geve_v1.gtf', sep = '\t', skiprows = 5, dtype = {0:'str', 7:'str'}, names = columns ) #tego typu 7 nie jestem pewna

In [4]:
def get_gene_id(df):
    df = df.copy()
    pattern = r'gene_id "(.*?)"'
    df['gene_id'] = df['attribute'].str.extract(pattern)
    return df 

In [5]:
ensembl_gtf = get_gene_id(ensembl_gtf)

In [6]:
file = open('TET2_ERV_names.txt')

overexpressed_ERVs = file.read() 
overexpressed_ERVs_list = overexpressed_ERVs.split('\n')[:-1]

In [7]:
ensembl_gtf_ERVs = ensembl_gtf[ensembl_gtf['gene_id'].isin(overexpressed_ERVs_list)]

In [8]:
rsem_m1_overexpr = rsem_m1[rsem_m1['gene_id'].isin(overexpressed_ERVs_list)]
rsem_m2_overexpr = rsem_m2[rsem_m2['gene_id'].isin(overexpressed_ERVs_list)]
rsem_m3_overexpr = rsem_m3[rsem_m3['gene_id'].isin(overexpressed_ERVs_list)]
rsem_m4_overexpr = rsem_m4[rsem_m4['gene_id'].isin(overexpressed_ERVs_list)]
rsem_m5_overexpr = rsem_m5[rsem_m5['gene_id'].isin(overexpressed_ERVs_list)]

In [9]:
print(rsem_m5_overexpr)

                                  gene_id                   transcript_id(s)  \
62838      Mmus38.chr11.6752021.6752656.-     Mmus38.chr11.6752021.6752656.-   
84706   Mmus38.chr4.101865756.101866367.-  Mmus38.chr4.101865756.101866367.-   
85297   Mmus38.chr4.141851853.141852761.-  Mmus38.chr4.141851853.141852761.-   
86438     Mmus38.chr4.46582665.46583417.+    Mmus38.chr4.46582665.46583417.+   
89049     Mmus38.chr5.44135940.44137139.+    Mmus38.chr5.44135940.44137139.+   
100384    Mmus38.chr9.48522394.48523164.-    Mmus38.chr9.48522394.48523164.-   

        length  effective_length  expected_count     TPM   FPKM  
62838    636.0             561.6           50.80   28.57  19.93  
84706    612.0             537.6            0.00    0.00   0.00  
85297    909.0             834.6            3.14    1.19   0.83  
86438    753.0             678.6            6.00    2.79   1.95  
89049   1200.0            1125.6           23.68    6.65   4.64  
100384   771.0             696.6          2

In [16]:

mouse_list = [rsem_m1_overexpr, rsem_m2_overexpr, rsem_m3_overexpr, rsem_m4_overexpr, rsem_m5_overexpr]
#columns_to_drop = ['transcript_id(s)', 'length', 'effective_length', 'TPM', 'FPKM']
columns_to_drop = ['transcript_id(s)', 'length', 'effective_length', 'expected_count', 'FPKM']

ctr = 1
full_df = pd.DataFrame()

ctr_to_name = {1:'control_1', 2:'control_2', 3:'mutated_1', 4:'mutated_2', 5: 'mutated_3'}

for mouse in mouse_list:
    mouse = mouse.copy()
    mouse['patient_id'] = ctr_to_name[ctr]
    mouse.drop(columns=columns_to_drop, inplace=True)
    #print(mouse)
    full_df = pd.concat([full_df, mouse], ignore_index=True)
    ctr += 1

print(full_df)


                              gene_id     TPM patient_id
0      Mmus38.chr11.6752021.6752656.-    3.53  control_1
1   Mmus38.chr4.101865756.101866367.-   18.60  control_1
2   Mmus38.chr4.141851853.141852761.-   15.19  control_1
3     Mmus38.chr4.46582665.46583417.+   32.42  control_1
4     Mmus38.chr5.44135940.44137139.+    1.29  control_1
5     Mmus38.chr9.48522394.48523164.-  362.25  control_1
6      Mmus38.chr11.6752021.6752656.-    2.43  control_2
7   Mmus38.chr4.101865756.101866367.-   26.45  control_2
8   Mmus38.chr4.141851853.141852761.-   17.15  control_2
9     Mmus38.chr4.46582665.46583417.+   13.83  control_2
10    Mmus38.chr5.44135940.44137139.+    0.00  control_2
11    Mmus38.chr9.48522394.48523164.-  583.89  control_2
12     Mmus38.chr11.6752021.6752656.-    0.00  mutated_1
13  Mmus38.chr4.101865756.101866367.-   14.20  mutated_1
14  Mmus38.chr4.141851853.141852761.-    3.74  mutated_1
15    Mmus38.chr4.46582665.46583417.+    8.98  mutated_1
16    Mmus38.chr5.44135940.4413

In [17]:
def extract_chrom(df):
    pattern = r"^Mmus38\.(chr[^.]+)\."
    df['chrom'] = df['gene_id'].str.extract(pattern)
    return df
    
def extract_start_pos(df):
    pattern = r"^Mmus38\.[^.]+\.([^.]+)\."
    df['start'] = df['gene_id'].str.extract(pattern)
    df.astype({'start': 'int32'}).dtypes
    return df

In [18]:
full_df = extract_chrom(full_df)
full_df = extract_start_pos(full_df)
print(full_df)

                              gene_id     TPM patient_id  chrom      start
0      Mmus38.chr11.6752021.6752656.-    3.53  control_1  chr11    6752021
1   Mmus38.chr4.101865756.101866367.-   18.60  control_1   chr4  101865756
2   Mmus38.chr4.141851853.141852761.-   15.19  control_1   chr4  141851853
3     Mmus38.chr4.46582665.46583417.+   32.42  control_1   chr4   46582665
4     Mmus38.chr5.44135940.44137139.+    1.29  control_1   chr5   44135940
5     Mmus38.chr9.48522394.48523164.-  362.25  control_1   chr9   48522394
6      Mmus38.chr11.6752021.6752656.-    2.43  control_2  chr11    6752021
7   Mmus38.chr4.101865756.101866367.-   26.45  control_2   chr4  101865756
8   Mmus38.chr4.141851853.141852761.-   17.15  control_2   chr4  141851853
9     Mmus38.chr4.46582665.46583417.+   13.83  control_2   chr4   46582665
10    Mmus38.chr5.44135940.44137139.+    0.00  control_2   chr5   44135940
11    Mmus38.chr9.48522394.48523164.-  583.89  control_2   chr9   48522394
12     Mmus38.chr11.67520

In [22]:
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import seaborn as sns
import os

def merge_images(image_files, output_filename, subplots_per_row, main_title):
    images = [Image.open(img) for img in image_files]
    widths, heights = zip(*(img.size for img in images))

    # Calculate the dimensions of the grid
    num_images = len(images)
    rows = (num_images - 1) // subplots_per_row + 1

    # Calculate the total width and height of the merged image
    max_width = max(widths) * subplots_per_row
    total_height = sum(heights[i] for i in range(0, num_images, subplots_per_row))
    title_height = 100  # Space for the title

    new_image = Image.new('RGB', (max_width, total_height + title_height), color=(255, 255, 255))

    # Draw the title
    draw = ImageDraw.Draw(new_image)
    try:
        title_font = ImageFont.truetype("Roboto-Regular.ttf", 40)  # Adjust the font size as needed
    except IOError:
        title_font = ImageFont.load_default()  # Load default font if the specified font is not found

    font_size = 60

    # Calculate font size for the default font
    while True:
        try:
            temp_font = ImageFont.truetype("Roboto-Regular.ttf", font_size)
        except IOError:
            temp_font = ImageFont.load_default()
        title_bbox = draw.textbbox((0, 0), main_title, font=temp_font)
        title_width = title_bbox[2] - title_bbox[0]
        if title_width < max_width or font_size <= 10:
            title_font = temp_font
            break
        font_size -= 1

    title_position = ((max_width - title_width) // 2, 20)  # Centered title
    draw.text(title_position, main_title, fill="black", font=title_font)

    y_offset = title_height + 40  # Add some padding for the title
    for row in range(0, num_images, subplots_per_row):
        max_height_in_row = max(heights[row:row+subplots_per_row])
        x_offset = 0
        for img in images[row:row+subplots_per_row]:
            new_image.paste(img, (x_offset, y_offset))
            x_offset += img.size[0]
        y_offset += max_height_in_row

    new_image.save(output_filename)

def visualise_ERV_expression_groups_3(df, filename, threshold=True, split_by_chrom=False, subplots_per_row=2, main_title="ERV Expression Levels"):
    palette = {'control_1': 'indianred', 'control_2': 'indianred', 'mutated_1': 'steelblue', 'mutated_2': 'steelblue', 'mutated_3': 'steelblue'}

    if split_by_chrom:
        # Group data by chromosome
        chrom_groups = df.groupby('chrom')
        num_chroms = len(chrom_groups)

        image_files = []

        for chrom, group in chrom_groups:
            # Sort the group by ascending order of 'start' position
            group['start'] = group['start'].astype(int)
            group_sorted = group.sort_values(by='start')
            fig, ax = plt.subplots(figsize=(15, 10))
            sns.barplot(data=group_sorted, x='gene_id', y='TPM', hue='patient_id', palette=palette, ax=ax)
            ax.set_title(f'Chromosome {chrom}', fontdict = {'fontsize':20})
            ax.set_xlabel('ERV id')
            ax.set_ylabel('TPM')
            ax.legend(title='Cell ID', loc='upper right')
            if threshold:
                ax.set_xticklabels(ax.get_xticklabels(), rotation=10)

            # Save each subplot individually
            img_filename = f'{filename}_{chrom}.png'
            plt.savefig(img_filename)
            plt.close()  # Close the current figure to prevent memory issues
            
            image_files.append(img_filename)

        # Merge the saved images into a single grid
        merge_images(image_files, f'{filename}_merged.png', subplots_per_row, main_title)

        # Remove individual subplot images
        for img_file in image_files:
            os.remove(img_file)

    else:
        sns.barplot(data=df, x='gene_id', y='TPM', hue='patient_id', palette=palette)
        plt.xlabel('ERV id')
        plt.ylabel('Count')
        plt.title(main_title)
        plt.legend(title='Cell ID', loc='upper right')
        if threshold:
            plt.xticks(rotation=90)
        plt.savefig(filename)
        plt.show()

# Example usage
visualise_ERV_expression_groups_3(full_df, 'TET2_diffexp_ERV', subplots_per_row=2, split_by_chrom=True, main_title="TET2: TPM values of differentially expressed ERV's")


/tmp/ipykernel_1235577/814984421.py:78: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=10)
/tmp/ipykernel_1235577/814984421.py:78: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=10)
/tmp/ipykernel_1235577/814984421.py:78: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=10)
/tmp/ipykernel_1235577/814984421.py:78: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=10)


In [ ]:
from PIL import Image

def merge_images(image_files, output_filename, subplots_per_row):
    images = [Image.open(img) for img in image_files]
    widths, heights = zip(*(img.size for img in images))

    # Calculate the dimensions of the grid
    num_images = len(images)
    rows = (num_images - 1) // subplots_per_row + 1

    # Calculate the total width and height of the merged image
    max_width = max(widths) * subplots_per_row
    total_height = sum(heights[i] for i in range(0, num_images, subplots_per_row))

    new_image = Image.new('RGB', (max_width, total_height), color=(255, 255, 255))

    y_offset = 0
    for row in range(0, num_images, subplots_per_row):
        max_height_in_row = max(heights[row:row+subplots_per_row])
        x_offset = 0
        for img in images[row:row+subplots_per_row]:
            new_image.paste(img, (x_offset, y_offset))
            x_offset += img.size[0]
        y_offset += max_height_in_row
    new_image.text((24,24), 
    new_image.save(output_filename)

def visualise_ERV_expression_gropus_3(df, filename, threshold=True, split_by_chrom=False, subplots_per_row=3):
    palette = {1: 'indianred', 2: 'indianred', 3: 'steelblue', 4: 'steelblue',5:'steelblue'}

    if split_by_chrom:
        # Group data by chromosome
        chrom_groups = df.groupby('chrom')
        num_chroms = len(chrom_groups)

        image_files = []

        for chrom, group in chrom_groups:
            # Sort the group by ascending order of 'start' position
            group['start'] = group['start'].astype(int)

            group_sorted = group.sort_values(by='start')
            print(group_sorted)
            fig, ax = plt.subplots(figsize=(15, 10))
            sns.barplot(data=group_sorted, x='gene_id', y='TPM', hue='patient_id', palette=palette, ax=ax)
            ax.set_title(f'Chromosome {chrom}')
            ax.set_xlabel('ERV id')
            ax.set_ylabel('TPM')
            ax.legend(title='Cell ID', loc='upper right')
            if threshold:
                ax.set_xticklabels(ax.get_xticklabels(), rotation=10)

            # Save each subplot individually
            img_filename = f'{filename}_{chrom}.png'
            plt.savefig(img_filename)
            plt.close()  # Close the current figure to prevent memory issues
            
            image_files.append(img_filename)

        # Merge the saved images into a single grid
        merge_images(image_files, f'{filename}_merged.png', subplots_per_row)

        # Remove individual subplot images
        for img_file in image_files:
            os.remove(img_file)

    else:    
        sns.barplot(data=df, x='gene_id', y='TPM', hue='patient_id', palette=palette)
        plt.xlabel('ERV id')
        plt.ylabel('Count')
        plt.title('ERV expression levels measured with RSEM')
        plt.legend(title='Cell ID', loc='upper right')
        if threshold:
            plt.xticks(rotation=90)
        plt.savefig(filename)
        plt.show()


In [ ]:
visualise_ERV_expression_gropus_3(full_df, 'TET2_diffexp_ERV', split_by_chrom = True, subplots_per_row = 2)

In [24]:
rsem_erv_1 = rsem_m1[rsem_m1['gene_id'].str.startswith('Mmus38')]
rsem_erv_2 = rsem_m2[rsem_m2['gene_id'].str.startswith('Mmus38')]
rsem_erv_3 = rsem_m3[rsem_m3['gene_id'].str.startswith('Mmus38')]
rsem_erv_4 = rsem_m4[rsem_m4['gene_id'].str.startswith('Mmus38')]
rsem_erv_5 = rsem_m5[rsem_m5['gene_id'].str.startswith('Mmus38')]

print(len(rsem_erv_1)) #in DEseq2 we have the same nr

60983


In [25]:
def count_breadth(expression_df):
    breadth = len(expression_df[expression_df['TPM'] > 1] )    
    return breadth

def count_abundance(expression_df):
    abundance = expression_df['TPM'].sum()
    return abundance

#def count_expected_count(expression_df):
#    expected_count = expression_df['expected_count'].sum()
#    return expected_count

In [26]:
#M1
m1_breadth = count_breadth(rsem_erv_1)
m1_abundance = count_abundance(rsem_erv_1)
#m1_expected_count = count_expected_count(rsem_erv_1)
#M2
m2_breadth = count_breadth(rsem_erv_2)
m2_abundance = count_abundance(rsem_erv_2)
#m2_expected_count = count_expected_count(rsem_erv_2)
#M3
m3_breadth = count_breadth(rsem_erv_3)
m3_abundance = count_abundance(rsem_erv_3)
#m3_expected_count = count_expected_count(rsem_erv_3)
#M4
m4_breadth = count_breadth(rsem_erv_4)
m4_abundance = count_abundance(rsem_erv_4)
#m4_expected_count = count_expected_count(rsem_erv_4)
#M5
m5_breadth = count_breadth(rsem_erv_5)
m5_abundance = count_abundance(rsem_erv_5)
#m5_expected_count = count_expected_count(rsem_erv_5)

In [27]:
print('m1 breadth : {} m1 abundance {} '.format(m1_breadth, m1_abundance))
print('m2 breadth : {} m2 abundance {} '.format(m2_breadth, m2_abundance))
print('m3 breadth : {} m3 abundance {} '.format(m3_breadth, m3_abundance))
print('m4 breadth : {} m4 abundance {} '.format(m4_breadth, m4_abundance))
print('m5 breadth : {} m5 abundance {} '.format(m5_breadth, m5_abundance))



m1 breadth : 564 m1 abundance 9250.670000000002 
m2 breadth : 661 m2 abundance 7443.51 
m3 breadth : 294 m3 abundance 2893.74 
m4 breadth : 384 m4 abundance 3969.85 
m5 breadth : 390 m5 abundance 3256.4999999999995 


TEcounts PART

In [28]:

def get_TEtranscript_path(directory_path, subfolder):
    subfolder_path = os.path.join(directory_path, subfolder)
    files = os.listdir(subfolder_path)
    if len(files) != 1:
        return 0
    return os.path.join(subfolder_path, files[0])
    
def modify_columns(columns, patient_id):
    modified_columns = [col + '_' + patient_id for col in columns]
    return modified_columns

def consolidate_gene_id_columns(df):
    gene_id_columns = [col for col in df.columns if col.startswith('gene_id_')]
    if gene_id_columns:
        df['gene_id'] = df[gene_id_columns[0]]
        df.drop(gene_id_columns, axis=1, inplace=True)
    return df

    
def load_TEtranscript_data(directory_path):
    columns = ['gene_id', 'count']
    TEtranscript_data = {}
    for subfolder in os.listdir(directory_path):
        patient_id, _ = subfolder.split('_')
        TE_file_path = get_TEtranscript_path(directory_path, subfolder)
        if TE_file_path == 0:
            print('couldn \'t load the TEfile')
            return 0
        TEtranscript_data[patient_id] = pd.read_csv(TE_file_path, sep='\t', header=0, names=modify_columns(columns, patient_id))
    return TEtranscript_data

# Example usage:
directory_path = '/work/iku/exp1019-cancer-epigenetics-and-ervs/01_BS_and_ERV/results/03_TEcount/TET2/'

TEtranscript_data = load_TEtranscript_data(directory_path)

In [29]:
print(TEtranscript_data)

{'1':                         gene_id_1  count_1
0              ENSMUSG00000000001        0
1              ENSMUSG00000000003        0
2              ENSMUSG00000000028       88
3              ENSMUSG00000000031        0
4              ENSMUSG00000000037        0
...                           ...      ...
117812     Zaphod2:hAT-Tip100:DNA        3
117813     Zaphod3:hAT-Tip100:DNA       39
117814      Zaphod:hAT-Tip100:DNA       66
117815         hAT-N1_Mam:hAT:DNA       14
117816  nhAT5a_ML:hAT-Charlie:DNA        0

[117817 rows x 2 columns], '3':                         gene_id_3  count_3
0              ENSMUSG00000000001        0
1              ENSMUSG00000000003        0
2              ENSMUSG00000000028      153
3              ENSMUSG00000000031        0
4              ENSMUSG00000000037        0
...                           ...      ...
117812     Zaphod2:hAT-Tip100:DNA        1
117813     Zaphod3:hAT-Tip100:DNA       54
117814      Zaphod:hAT-Tip100:DNA      114
117815         

CONCATENATE TEtranscript DATA INTO ONE BIG DATAFRAME

In [ ]:
def concatenate_patient_dataframes(TEtranscript_data):
    patient_dfs = list(TEtranscript_data.values())
    concatenated_df = pd.concat(patient_dfs, axis=1) if patient_dfs else None
    concatenated_df = consolidate_gene_id_columns(concatenated_df)
    return concatenated_df

In [ ]:
TET2_TE_df = concatenate_patient_dataframes(TEtranscript_data)
print(TET2_TE_df)

FILTER OUT ALL NON ERV DATA

In [ ]:
TET2_TE_df_ERV = TET2_TE_df[TET2_TE_df['gene_id'].str.startswith('Mmus38')]
#print(TET2_TE_df_ERV.head())

TET2_TE_df_filt = TET2_TE_df_ERV[TET2_TE_df_ERV['gene_id'].isin(overexpr_genes)]
print(TET2_TE_df_filt)

FILTER ALL ERVS THAT ARE NOT EXPRESSED IN ANY OF THE MICE

In [ ]:
def filter_non_expressed_ERV(df):
    count_columns = [col for col in df.columns if col.startswith('count')]
    
    # Sum values along the rows for selected columns
    df.loc[:, 'sum_counts'] = df[count_columns].astype(int).sum(axis=1)
    
    # Filter rows where sum_counts is greater than 0
    df = df.loc[df['sum_counts'] > 0]
    
    # Drop the temporary 'sum_counts' column
    del df['sum_counts']
    
    return df

In [ ]:
TET2_TE_df_ERV_filtered = filter_non_expressed_ERV(TET2_TE_df_ERV)
print(TET2_TE_df_ERV_filtered.shape)

VISUALISE DIFFERENCES IN ERV EXPRESSION BETWEEN THE SAMPLES

In [ ]:
def reshape_dataframe(df):
    id_vars = ['gene_id']
    value_vars = [col for col in df.columns if col.startswith('count_')]
    
    # Melt the DataFrame
    melted_df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='patient_id', value_name='count')
    
    # Extract patient_id from column names
    melted_df['patient_id'] = melted_df['patient_id'].str.split('_', expand=True)[1].astype(int)
    
    return melted_df

In [ ]:
reshaped = reshape_dataframe(TET2_TE_df_ERV_filtered)
print(reshaped.head())
def visualise_ERV_expression(df, plot_type, threshold = False):
    plt.figure(figsize=(30, 30)) 

    if plot_type == 'scatter':
        sns.scatterplot(data=df, x='gene_id', y='count', hue='patient_id', palette = 'tab10') #s=4
    if plot_type == 'line':
        sns.lineplot(data=df, x='gene_id', y='count', hue='patient_id', palette = 'tab10')
    
    plt.xlabel('ERV id')
    plt.ylabel('count')

    plt.title('ERV expression levels measured with TEtranscripts')

    # Adding legend
    plt.legend(title='Mouse ID', loc='upper right')
    if threshold == True:
        plt.xticks(rotation=90)
    plt.savefig('TEcounts_TET2_threshold.jpg')
    plt.show()    


In [ ]:
# plot_type 'line; or 'scatter'
visualise_ERV_expression(reshaped, 'line')

In [ ]:
def visualise_ERV_expression_gropus(df, plot_type, threshold = False):
    plt.figure(figsize=(30, 30)) 

    palette = {1: 'red', 2: 'red', 3: 'blue', 4: 'blue', 5: 'blue'}

    if plot_type == 'scatter':
        sns.scatterplot(data=df, x='gene_id', y='count', hue='patient_id', palette = palette) #s=4
    if plot_type == 'line':
        sns.lineplot(data=df, x='gene_id', y='count', hue='patient_id', palette = palette)
    
    plt.xlabel('ERV id')
    plt.ylabel('count')

    plt.title('ERV expression levels measured with TEtranscripts')

    # Adding legend
    plt.legend(title='Mouse ID', loc='upper right')

    if threshold == True:
        plt.xticks(rotation=90)
    plt.savefig('TET2_threshold.jpg')
    plt.show()    

import cytosine reports from trimmed and untrimmed data 

In [ ]:
visualise_ERV_expression_gropus(reshaped, 'line')

ONLY VISUALISE ERV's WITH EXPRESSION HIGHER THAN A THRESHOLD

In [ ]:
def remove_lowly_expressed(df, threshold):
    counts_cols = [col for col in df.columns if col.startswith('count_')]
    mask = (df[counts_cols] >= threshold).any(axis=1)
    filtered_df = df[mask]
    return filtered_df

In [ ]:
highly_expressed = remove_lowly_expressed(TET2_TE_df_ERV_filtered, 500)
highly_expressed_reshaped = reshape_dataframe(highly_expressed)
print(highly_expressed)

In [ ]:
#visualise_ERV_expression_gropus(highly_expressed_reshaped, 'scatter', threshold = True)
visualise_ERV_expression(highly_expressed_reshaped, 'line', threshold = True)


HIGHLY EXPRESSED ERVS IN TET2 MOUSES

In [ ]:
highly_expressed_ERVs = list(highly_expressed['gene_id'])
print(highly_expressed_ERVs)

In [ ]:
for gene in overexpr_genes:
    print(highly_expressed[highly_expressed['gene_id'] == gene])
    

In [ ]:
from PIL import Image

def merge_images(image_files, output_filename, subplots_per_row):
    images = [Image.open(img) for img in image_files]
    widths, heights = zip(*(img.size for img in images))

    # Calculate the dimensions of the grid
    num_images = len(images)
    rows = (num_images - 1) // subplots_per_row + 1

    # Calculate the total width and height of the merged image
    max_width = max(widths) * subplots_per_row
    total_height = sum(heights[i] for i in range(0, num_images, subplots_per_row))

    new_image = Image.new('RGB', (max_width, total_height), color=(255, 255, 255))

    y_offset = 0
    for row in range(0, num_images, subplots_per_row):
        max_height_in_row = max(heights[row:row+subplots_per_row])
        x_offset = 0
        for img in images[row:row+subplots_per_row]:
            new_image.paste(img, (x_offset, y_offset))
            x_offset += img.size[0]
        y_offset += max_height_in_row

    new_image.save(output_filename)

def visualise_ERV_expression_gropus_3(df, filename, threshold=True, split_by_chrom=False, subplots_per_row=3):
    palette = {1: 'indianred', 2: 'indianred', 3: 'steelblue', 4: 'steelblue',5:'steelblue'}

    if split_by_chrom:
        # Group data by chromosome
        chrom_groups = df.groupby('chrom')
        num_chroms = len(chrom_groups)

        image_files = []

        for chrom, group in chrom_groups:
            # Sort the group by ascending order of 'start' position
            group['start'] = group['start'].astype(int)

            group_sorted = group.sort_values(by='start')
            print(group_sorted)
            fig, ax = plt.subplots(figsize=(15, 10))
            sns.barplot(data=group_sorted, x='gene_id', y='TPM', hue='patient_id', palette=palette, ax=ax)
            ax.set_title(f'Chromosome {chrom}')
            ax.set_xlabel('ERV id')
            ax.set_ylabel('TPM')
            ax.legend(title='Cell ID', loc='upper right')
            if threshold:
                ax.set_xticklabels(ax.get_xticklabels(), rotation=10)

            # Save each subplot individually
            img_filename = f'{filename}_{chrom}.png'
            plt.savefig(img_filename)
            plt.close()  # Close the current figure to prevent memory issues
            
            image_files.append(img_filename)

        # Merge the saved images into a single grid
        merge_images(image_files, f'{filename}_merged.png', subplots_per_row)

        # Remove individual subplot images
        for img_file in image_files:
            os.remove(img_file)

    else:    
        sns.barplot(data=df, x='gene_id', y='TPM', hue='patient_id', palette=palette)
        plt.xlabel('ERV id')
        plt.ylabel('Count')
        plt.title('ERV expression levels measured with RSEM')
        plt.legend(title='Cell ID', loc='upper right')
        if threshold:
            plt.xticks(rotation=90)
        plt.savefig(filename)
        plt.show()


In [ ]:
Dnmt3a_form = {
    'GSM2309133' : '+/-',
    'GSM2309134' : '+/-',
    'GSM2309135' : '+/+',
    'GSM2309136' : '+/+',
    'GSM2309137' : '+/m',
    'GSM2309138' : '+/+',
    'GSM2309139' : '+/m',
    'GSM2309140' : '+/m',
    'GSM2309141' : '+/m'

}
#rnaseq sra id : patient_id, 'rnaseq sample', 'genotype', 'bissulfite sra
Tet2_form = {
    'SRR21643414' : [1, 'GSM6594851', '+/+', 'SRR21658932'],
    'SRR21643410' : [2, 'GSM6594852', '+/+', 'SRR21658928'],
    'SRR21643406' : [3, 'GSM6594853', '-,-', 'SRR21658920'],
    'SRR21643402' : [4, 'GSM6594854', '-/-', 'SRR21658916'],
    'SRR21643398' : [5, 'GSM6594855', '-/-', 'SRR21658912']
}
